In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [2]:
train = pd.read_csv('C:/Users/user/LikeLion_AI/data/CSV/DACON_Parking_Demand/train.csv')
test = pd.read_csv('C:/Users/user/LikeLion_AI/data/CSV/DACON_Parking_Demand/test.csv')
sub = pd.read_csv('C:/Users/user/LikeLion_AI/data/CSV/DACON_Parking_Demand/sample_submission.csv')
age = pd.read_csv('C:/Users/user/LikeLion_AI/data/CSV/DACON_Parking_Demand/age_gender_info.csv')

train.shape, test.shape, sub.shape, age.shape

((2952, 15), (1022, 14), (150, 2), (16, 23))

In [3]:
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   object 
 3   지역                            2952 non-null   object 
 4   공급유형                          2952 non-null   object 
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   object 
 9   임대보증금                         2383 non-null   object 
 10  임대료                           2383 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2741 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2948 non-null   float64
 13  단지내

(None, None)

In [4]:
train.isnull().sum(), test.isnull().sum()

(단지코드                              0
 총세대수                              0
 임대건물구분                            0
 지역                                0
 공급유형                              0
 전용면적                              0
 전용면적별세대수                          0
 공가수                               0
 자격유형                              0
 임대보증금                           569
 임대료                             569
 도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
 도보 10분거리 내 버스정류장 수                4
 단지내주차면수                           0
 등록차량수                             0
 dtype: int64,
 단지코드                              0
 총세대수                              0
 임대건물구분                            0
 지역                                0
 공급유형                              0
 전용면적                              0
 전용면적별세대수                          0
 공가수                               0
 자격유형                              2
 임대보증금                           180
 임대료                             180
 도보 10분거리 내 지하철역 수(환승노선

train1, test1 : 전용 면적 당 임대보증금과 임대료를 예측하기 위한 DataFrame

#### 그리고 오류가 있는 단지코드를 모두 삭제

In [100]:
train1 = train.copy()

drop_danji = ['C2085', 'C1397', 'C2431', 'C1649', 'C1036', 
              'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']    # 총 12개


print(len(train1['단지코드'].unique()))   # drop 이전 : 423
for code in drop_danji:
    train1 = train1[train1['단지코드'] != code]

print(len(train1['단지코드'].unique()))   # drop 이후 : 411
print()

print((train1[train1['임대건물구분']=='아파트']['임대료'] == '-').sum())
print(train1[train1['임대건물구분']=='아파트']['임대료'].isna().sum())

print((train1[train1['임대건물구분']=='상가']['임대료'] == '-').sum())
print(train1[train1['임대건물구분']=='상가']['임대료'].isna().sum())

423
411

15
7
0
562


In [101]:
test1 = test.copy()

test_drop = ['C2675', 'C2335', 'C1327']

for code in test_drop:
    test1 = test1[test1['단지코드'] != code]
    
print((test1[test1['임대건물구분']=='아파트']['임대료'] == '-').sum())
print(test1[test1['임대건물구분']=='아파트']['임대료'].isna().sum())

print((test1[test1['임대건물구분']=='상가']['임대료'] == '-').sum())
print(test1[test1['임대건물구분']=='상가']['임대료'].isna().sum())

6
3
0
177


### 단지 별 임대료 결측치 

In [102]:
danji = train['단지코드'].value_counts().index
danji_list = []

for code in danji:
    null_sum = train1[train1['단지코드']==code]['임대료'].isnull().sum() 
    null_sum += (train1[train1['단지코드']==code]['임대료'] == '-').sum()
    if null_sum:
        danji_list.append(code)
        print(f"{code}의 갯수 = {len(train1[train1['단지코드']==code])}")
        print(f'{code}의 임대료 결측치 갯수 = {null_sum}')
        print(train1[train1['단지코드']==code]['임대건물구분'].value_counts())
        print()

C1439의 갯수 = 48
C1439의 임대료 결측치 갯수 = 45
상가     45
아파트     3
Name: 임대건물구분, dtype: int64

C2132의 갯수 = 47
C2132의 임대료 결측치 갯수 = 43
상가     43
아파트     4
Name: 임대건물구분, dtype: int64

C1859의 갯수 = 40
C1859의 임대료 결측치 갯수 = 35
상가     35
아파트     5
Name: 임대건물구분, dtype: int64

C1790의 갯수 = 33
C1790의 임대료 결측치 갯수 = 30
상가     30
아파트     3
Name: 임대건물구분, dtype: int64

C2597의 갯수 = 32
C2597의 임대료 결측치 갯수 = 29
상가     29
아파트     3
Name: 임대건물구분, dtype: int64

C1874의 갯수 = 30
C1874의 임대료 결측치 갯수 = 27
상가     27
아파트     3
Name: 임대건물구분, dtype: int64

C2034의 갯수 = 29
C2034의 임대료 결측치 갯수 = 26
상가     26
아파트     3
Name: 임대건물구분, dtype: int64

C2289의 갯수 = 24
C2289의 임대료 결측치 갯수 = 20
상가     20
아파트     4
Name: 임대건물구분, dtype: int64

C2310의 갯수 = 23
C2310의 임대료 결측치 갯수 = 20
상가     20
아파트     3
Name: 임대건물구분, dtype: int64

C1109의 갯수 = 22
C1109의 임대료 결측치 갯수 = 20
상가     20
아파트     2
Name: 임대건물구분, dtype: int64

C1616의 갯수 = 22
C1616의 임대료 결측치 갯수 = 19
상가     19
아파트     3
Name: 임대건물구분, dtype: int64

C1875의 갯수 = 22
C1875의 임대료 결측치 갯수 = 19
상가     19
아파트   

In [103]:
danji = test['단지코드'].value_counts().index
test_danji = []

for code in danji:
    null_sum = test1[test1['단지코드']==code]['임대료'].isnull().sum() 
    null_sum += (test1[test1['단지코드']==code]['임대료'] == '-').sum()
    if null_sum:
        test_danji.append(code)
        print(f"{code}의 갯수 = {len(test1[test1['단지코드']==code])}")
        print(f'{code}의 임대료 결측치 갯수 = {null_sum}')
        print(test1[test1['단지코드']==code]['임대건물구분'].value_counts())
        print()

C2676의 갯수 = 46
C2676의 임대료 결측치 갯수 = 43
상가     43
아파트     3
Name: 임대건물구분, dtype: int64

C2253의 갯수 = 31
C2253의 임대료 결측치 갯수 = 28
상가     28
아파트     3
Name: 임대건물구분, dtype: int64

C1812의 갯수 = 27
C1812의 임대료 결측치 갯수 = 25
상가     25
아파트     2
Name: 임대건물구분, dtype: int64

C1006의 갯수 = 27
C1006의 임대료 결측치 갯수 = 23
상가     20
아파트     7
Name: 임대건물구분, dtype: int64

C1729의 갯수 = 25
C1729의 임대료 결측치 갯수 = 17
상가     17
아파트     8
Name: 임대건물구분, dtype: int64

C2177의 갯수 = 25
C2177의 임대료 결측치 갯수 = 22
상가     22
아파트     3
Name: 임대건물구분, dtype: int64

C2033의 갯수 = 25
C2033의 임대료 결측치 갯수 = 22
상가     22
아파트     3
Name: 임대건물구분, dtype: int64

C1267의 갯수 = 11
C1267의 임대료 결측치 갯수 = 4
아파트    11
Name: 임대건물구분, dtype: int64

C2152의 갯수 = 2
C2152의 임대료 결측치 갯수 = 2
아파트    2
Name: 임대건물구분, dtype: int64



In [104]:
test_danji

['C2676',
 'C2253',
 'C1812',
 'C1006',
 'C1729',
 'C2177',
 'C2033',
 'C1267',
 'C2152']

#### 결측치가 전체의 과반수를 차지하고 있는 상황이 대부분.. 어떻게 처리해야할까?

1. 우선 다 삭제하고 해본다.

2. 과반수 이하가 있는 곳만 평균값으로 살린다.

3. 전용면적을 비율로 임대료를 각각 계산한다.

4. 시세를 예측해서 진행해본다.

#### -> 뭔가 3번이 제일 끌리니까 3번으로 바로 해보자!

15평 -> 500/25(기준) 

100평 -> 3000/120(번화가) >> 임대아파트

400평 -> 10000/400(예상)

In [105]:
train1.loc[train1['단지코드']==danji_list[2], :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
261,C1859,2424,아파트,대전광역시,영구임대,26.37,1865,1.0,C,5787000,79980,1.0,4.0,487.0,116.0
262,C1859,2424,아파트,대전광역시,영구임대,26.37,10,1.0,C,5787000,79980,1.0,4.0,487.0,116.0
263,C1859,2424,아파트,대전광역시,영구임대,26.37,10,1.0,C,5787000,79980,1.0,4.0,487.0,116.0
264,C1859,2424,아파트,대전광역시,영구임대,31.32,478,1.0,C,6873000,94990,1.0,4.0,487.0,116.0
265,C1859,2424,아파트,대전광역시,영구임대,52.74,26,1.0,C,11574000,159960,1.0,4.0,487.0,116.0
266,C1859,2424,상가,대전광역시,임대상가,38.00,1,1.0,D,NaN,NaN,1.0,4.0,487.0,116.0
267,C1859,2424,상가,대전광역시,임대상가,38.00,1,1.0,D,NaN,NaN,1.0,4.0,487.0,116.0
268,C1859,2424,상가,대전광역시,임대상가,31.84,1,1.0,D,NaN,NaN,1.0,4.0,487.0,116.0
269,C1859,2424,상가,대전광역시,임대상가,31.84,1,1.0,D,NaN,NaN,1.0,4.0,487.0,116.0
270,C1859,2424,상가,대전광역시,임대상가,31.84,1,1.0,D,NaN,NaN,1.0,4.0,487.0,116.0


#### 단지 별로 임대건물구분, 공급유형, 전용면적, 전용면적별세대수, 자격유형, 임대보증급, 임대료 빼고 다 똑같구나..

#### 공가수 : 데이터 수집 당시 임차되고 있지 않은 빈 집

In [106]:
## 임대보증금과 임대료의 NaN 값과 - 값을 0으로 변환
train1.loc[train1['임대보증금'].isna(), '임대보증금'] = '0'
train1.loc[train1['임대료'].isna(), '임대료'] = '0'

train1.loc[train1['임대보증금']=='-', '임대보증금'] = '0'
train1.loc[train1['임대료']=='-', '임대료'] = '0'

train1['임대보증금'] = train1['임대보증금'].astype('float')
train1['임대료'] = train1['임대료'].astype('float')
train1 = train1.reset_index()

train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2869 entries, 0 to 2868
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   index                         2869 non-null   int64  
 1   단지코드                          2869 non-null   object 
 2   총세대수                          2869 non-null   int64  
 3   임대건물구분                        2869 non-null   object 
 4   지역                            2869 non-null   object 
 5   공급유형                          2869 non-null   object 
 6   전용면적                          2869 non-null   float64
 7   전용면적별세대수                      2869 non-null   int64  
 8   공가수                           2869 non-null   float64
 9   자격유형                          2869 non-null   object 
 10  임대보증금                         2869 non-null   float64
 11  임대료                           2869 non-null   float64
 12  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2662 non-null   float64
 13  도보 

In [107]:
## 임대보증금과 임대료의 NaN 값과 - 값을 0으로 변환
test1.loc[test1['임대보증금'].isna(), '임대보증금'] = '0'
test1.loc[test1['임대료'].isna(), '임대료'] = '0'

test1.loc[test1['임대보증금']=='-', '임대보증금'] = '0'
test1.loc[test1['임대료']=='-', '임대료'] = '0'

test1['임대보증금'] = test1['임대보증금'].astype('float')
test1['임대료'] = test1['임대료'].astype('float')
test1 = test1.reset_index()

test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   index                         1008 non-null   int64  
 1   단지코드                          1008 non-null   object 
 2   총세대수                          1008 non-null   int64  
 3   임대건물구분                        1008 non-null   object 
 4   지역                            1008 non-null   object 
 5   공급유형                          1008 non-null   object 
 6   전용면적                          1008 non-null   float64
 7   전용면적별세대수                      1008 non-null   int64  
 8   공가수                           1008 non-null   float64
 9   자격유형                          1006 non-null   object 
 10  임대보증금                         1008 non-null   float64
 11  임대료                           1008 non-null   float64
 12  도보 10분거리 내 지하철역 수(환승노선 수 반영)  970 non-null    float64
 13  도보 

### train

In [108]:
## 임대료에 결측치가 포함된 단지코드와 단위 면적당 임대료 및 임대보증금의 평균을 rent_dict에 저장
rent_dict = {}

for code in danji_list:
    temp_data = train1.loc[train1['단지코드']==code, :]
    for i in temp_data.index:
        if temp_data.loc[i, '임대료']:   # 임대료가 NaN이 아닌 경우
            deposit_list = []
            rental_list = []
            
            area = temp_data.loc[i, '전용면적']
            deposit = int(temp_data.loc[i, '임대보증금'])
            rental = int(temp_data.loc[i, '임대료'])
            
            deposit_list.append(1/area*deposit)
            rental_list.append(1/area*rental)
            
            deposit_mean = round(np.mean(deposit_list), 3)
            rental_mean = round(np.mean(rental_list), 3)
            
            rent_dict[code] = [deposit_mean, rental_mean]

In [109]:
rent_dict

{'C1439': [219444.444, 3032.986],
 'C2132': [148339.719, 2550.766],
 'C1859': [219453.925, 3032.992],
 'C1790': [119125.16, 2650.702],
 'C2597': [219444.444, 3032.886],
 'C1874': [119125.16, 2650.702],
 'C2034': [341379.31, 4395.275],
 'C2289': [219444.444, 3032.886],
 'C2310': [219444.444, 3032.886],
 'C1109': [219444.444, 3032.886],
 'C1616': [1340277.778, 697.917],
 'C1875': [119125.16, 2650.702],
 'C2190': [119125.16, 2650.702],
 'C2038': [219444.444, 3032.986],
 'C2135': [341379.31, 4395.275],
 'C2258': [219444.444, 3032.986],
 'C1004': [119123.205, 2650.794],
 'C2621': [219453.925, 3032.992],
 'C2109': [119127.297, 2650.919],
 'C1704': [219450.743, 3032.754],
 'C1775': [152919.948, 2838.911],
 'C1983': [308748.404, 4560.983],
 'C2212': [152905.492, 2838.761],
 'C2571': [115889.268, 2578.688],
 'C1350': [1059763.017, 2544.726],
 'C1056': [119127.297, 2650.919],
 'C1206': [119125.16, 2650.702],
 'C1039': [430776.201, 3552.401],
 'C1722': [281385.696, 4410.281],
 'C2416': [119127.29

In [110]:
train1.loc[train1['임대료']==0, :]

,index,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
80,88,C1925,601,상가,강원도,임대상가,32.10,1,9.0,D,0.0,0.0,0.0,4.0,117.0,75.0
81,89,C1925,601,상가,강원도,임대상가,32.10,1,9.0,D,0.0,0.0,0.0,4.0,117.0,75.0
82,90,C1925,601,상가,강원도,임대상가,32.10,1,9.0,D,0.0,0.0,0.0,4.0,117.0,75.0
83,91,C1925,601,상가,강원도,임대상가,72.16,1,9.0,D,0.0,0.0,0.0,4.0,117.0,75.0
93,101,C1874,619,상가,충청남도,임대상가,12.62,1,2.0,D,0.0,0.0,NaN,2.0,97.0,62.0
94,102,C1874,619,상가,충청남도,임대상가,17.40,1,2.0,D,0.0,0.0,NaN,2.0,97.0,62.0
95,103,C1874,619,상가,충청남도,임대상가,17.40,1,2.0,D,0.0,0.0,NaN,2.0,97.0,62.0
96,104,C1874,619,상가,충청남도,임대상가,22.89,1,2.0,D,0.0,0.0,NaN,2.0,97.0,62.0
97,105,C1874,619,상가,충청남도,임대상가,23.13,1,2.0,D,0.0,0.0,NaN,2.0,97.0,62.0
98,106,C1874,619,상가,충청남도,임대상가,23.13,1,2.0,D,0.0,0.0,NaN,2.0,97.0,62.0


In [111]:
pd.set_option('display.max_rows', None)
grouped = train1.groupby(['임대건물구분', '지역', '공급유형'])
grouped.get_group(('아파트', '부산광역시', '국민임대'))

,index,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
120,128,C2650,875,아파트,부산광역시,국민임대,36.44,557,13.0,A,11492000.0,104930.0,0.0,4.0,616.0,159.0
121,129,C2650,875,아파트,부산광역시,국민임대,45.09,20,13.0,A,14215000.0,141630.0,0.0,4.0,616.0,159.0
122,130,C2650,875,아파트,부산광역시,국민임대,45.32,298,13.0,A,14292000.0,141630.0,0.0,4.0,616.0,159.0
512,520,C1672,1350,아파트,부산광역시,국민임대,51.88,886,15.0,A,17417000.0,213320.0,0.0,1.0,1043.0,873.0
513,521,C1672,1350,아파트,부산광역시,국민임대,59.63,464,15.0,A,20018000.0,246330.0,0.0,1.0,1043.0,873.0
609,617,C1613,840,아파트,부산광역시,국민임대,51.49,220,7.0,A,20163000.0,255200.0,1.0,1.0,594.0,498.0
610,618,C1613,840,아파트,부산광역시,국민임대,51.95,454,7.0,A,20345000.0,257630.0,1.0,1.0,594.0,498.0
611,619,C1613,840,아파트,부산광역시,국민임대,59.64,166,7.0,A,23355000.0,298290.0,1.0,1.0,594.0,498.0
660,668,C1430,1533,아파트,부산광역시,국민임대,51.77,223,32.0,A,25966000.0,144800.0,0.0,3.0,1296.0,839.0
661,669,C1430,1533,아파트,부산광역시,국민임대,51.86,258,32.0,A,27465000.0,156030.0,0.0,3.0,1296.0,839.0


In [112]:
## C1326과 사정이 가장 비슷한 C1430의 전용면적당 임대료와 임대보증금을 가져오기로 함.
rent_dict['C1326'] = [560613.289, 3257.644]
rent_dict

{'C1439': [219444.444, 3032.986],
 'C2132': [148339.719, 2550.766],
 'C1859': [219453.925, 3032.992],
 'C1790': [119125.16, 2650.702],
 'C2597': [219444.444, 3032.886],
 'C1874': [119125.16, 2650.702],
 'C2034': [341379.31, 4395.275],
 'C2289': [219444.444, 3032.886],
 'C2310': [219444.444, 3032.886],
 'C1109': [219444.444, 3032.886],
 'C1616': [1340277.778, 697.917],
 'C1875': [119125.16, 2650.702],
 'C2190': [119125.16, 2650.702],
 'C2038': [219444.444, 3032.986],
 'C2135': [341379.31, 4395.275],
 'C2258': [219444.444, 3032.986],
 'C1004': [119123.205, 2650.794],
 'C2621': [219453.925, 3032.992],
 'C2109': [119127.297, 2650.919],
 'C1704': [219450.743, 3032.754],
 'C1775': [152919.948, 2838.911],
 'C1983': [308748.404, 4560.983],
 'C2212': [152905.492, 2838.761],
 'C2571': [115889.268, 2578.688],
 'C1350': [1059763.017, 2544.726],
 'C1056': [119127.297, 2650.919],
 'C1206': [119125.16, 2650.702],
 'C1039': [430776.201, 3552.401],
 'C1722': [281385.696, 4410.281],
 'C2416': [119127.29

### test

In [113]:
## 임대료에 결측치가 포함된 단지코드와 단위 면적당 임대료 및 임대보증금의 평균을 test_rent에 저장
test_rent = {}

for code in test_danji:
    temp_data = test1.loc[test1['단지코드']==code, :]
    for i in temp_data.index:
        if temp_data.loc[i, '임대료']:   # 임대료가 NaN이 아닌 경우
            deposit_list = []
            rental_list = []
            
            area = temp_data.loc[i, '전용면적']
            deposit = int(temp_data.loc[i, '임대보증금'])
            rental = int(temp_data.loc[i, '임대료'])
            
            deposit_list.append(1/area*deposit)
            rental_list.append(1/area*rental)
            
            deposit_mean = round(np.mean(deposit_list), 3)
            rental_mean = round(np.mean(rental_list), 3)
            
            test_rent[code] = [deposit_mean, rental_mean]

In [114]:
## C2152단지는 아파트 2개가 끝이고 둘다 결측치라 같은 강원도이고 영구임대인 아파트단지(C1729)의 값을 가져오기로 함
test_rent['C2152'] = [152919.021, 2838.983]
test_rent

{'C2676': [219444.444, 3032.886],
 'C2253': [119125.16, 2650.702],
 'C1812': [148754.789, 2761.814],
 'C1006': [219444.444, 3032.886],
 'C1729': [152919.021, 2838.983],
 'C2177': [119125.16, 2650.702],
 'C2033': [152901.024, 2838.832],
 'C1267': [475765.529, 3903.325],
 'C2152': [152919.021, 2838.983]}

```python
for code, value in rent_dict.items():
    temp_data = train1.loc[train1['단지코드']==code, :]
    for i in temp_data.index:
        if not temp_data.loc[i, '임대료']:   # 임대료가 빈 경우
            temp_data['임대보증금'] = value[0] * temp_data.loc[i, '전용면적']
            temp_data['임대료'] = value[1] * temp_data.loc[i, '전용면적']
```

DataFrame 접근방식이 틀려서 이렇게 변경이 안되는듯

### train

In [115]:
## 결측치가 포함된 단지코드의 단위면적당 임대료/임대보증금과 전용면적을 곱한 값을 기존의 임대보증금과 임대료의 수치에 덮어씌움
for code, value in rent_dict.items():
    train1.loc[train1['단지코드']==code, '임대보증금'] = value[0] * train1.loc[train1['단지코드']==code]['전용면적']
    train1.loc[train1['단지코드']==code, '임대료'] = value[1] * train1.loc[train1['단지코드']==code]['전용면적']

In [116]:
## 상가는 아파트와 시세가 다르기 때문에 같은 면적 대비 임대보증금을 1.2배 해주고 임대료를 10배 해준다.
for code in rent_dict.keys():
    train1.loc[(train1['단지코드']==code) & (train1['임대건물구분']=='상가'), '임대보증금']\
    = train1.loc[(train1['단지코드']==code) & (train1['임대건물구분']=='상가')]['임대보증금'] * 1.2
    
    train1.loc[(train1['단지코드']==code) & (train1['임대건물구분']=='상가'), '임대료']\
    = train1.loc[(train1['단지코드']==code) & (train1['임대건물구분']=='상가')]['임대료'] * 10

In [117]:
## 결측치 확인
print((train1[train1['임대건물구분']=='아파트']['임대료'] == '-').sum())
print(train1[train1['임대건물구분']=='아파트']['임대료'].isna().sum())
print((train1[train1['임대건물구분']=='아파트']['임대료'] == 0).sum())
print((train1[train1['임대건물구분']=='아파트']['임대보증금'] == 0).sum())

print((train1[train1['임대건물구분']=='상가']['임대료'] == '-').sum())
print(train1[train1['임대건물구분']=='상가']['임대료'].isna().sum())
print((train1[train1['임대건물구분']=='상가']['임대료'] == 0).sum())
print((train1[train1['임대건물구분']=='상가']['임대보증금'] == 0).sum())

0
0
0
0
0
0
0
0


### test

In [118]:
## 결측치가 포함된 단지코드의 단위면적당 임대료/임대보증금과 전용면적을 곱한 값을 기존의 임대보증금과 임대료의 수치에 덮어씌움
for code, value in test_rent.items():
    test1.loc[test1['단지코드']==code, '임대보증금'] = value[0] * test1.loc[test1['단지코드']==code]['전용면적']
    test1.loc[test1['단지코드']==code, '임대료'] = value[1] * test1.loc[test1['단지코드']==code]['전용면적']

In [119]:
## 상가는 아파트와 시세가 다르기 때문에 같은 면적 대비 임대보증금을 1.2배 해주고 임대료를 10배 해준다.
for code in test_rent.keys():
    test1.loc[(test1['단지코드']==code) & (test1['임대건물구분']=='상가'), '임대보증금']\
    = test1.loc[(test1['단지코드']==code) & (test1['임대건물구분']=='상가')]['임대보증금'] * 1.2
    
    test1.loc[(test1['단지코드']==code) & (test1['임대건물구분']=='상가'), '임대료']\
    = test1.loc[(test1['단지코드']==code) & (test1['임대건물구분']=='상가')]['임대료'] * 10

In [120]:
## 결측치 확인
print((test1[test1['임대건물구분']=='아파트']['임대료'] == '-').sum())
print(test1[test1['임대건물구분']=='아파트']['임대료'].isna().sum())
print((test1[test1['임대건물구분']=='아파트']['임대료'] == 0).sum())
print((test1[test1['임대건물구분']=='아파트']['임대보증금'] == 0).sum())

print((test1[test1['임대건물구분']=='상가']['임대료'] == '-').sum())
print(test1[test1['임대건물구분']=='상가']['임대료'].isna().sum())
print((test1[test1['임대건물구분']=='상가']['임대료'] == 0).sum())
print((test1[test1['임대건물구분']=='상가']['임대보증금'] == 0).sum())

0
0
0
0
0
0
0
0


#### 지하철 역 결측치가 있는 단지코드 출력

In [75]:
train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()]['단지코드'].value_counts()

C1874    30
C1616    22
C1875    22
C2258    20
C1004    19
C1704    17
C1983    15
C1350    13
C2583    11
C2644     7
C2100     6
C1068     6
C1312     4
C1005     3
C1424     3
C1175     3
C2156     3
C2216     2
C2520     1
Name: 단지코드, dtype: int64

#### 위의 단지코드에는 지하철 역 값이 전혀 없음.

1. 아예 삭제한다.

2. 최빈값인 0을 대입한다.

3. 결측치가 있는 각 단지의 등록차량수를 평균내어 지하철 역 값에 따른 등록차량수의 평균과 비교해 제일 비슷한 값으로 대입한다.

In [22]:
train1.groupby('도보 10분거리 내 지하철역 수(환승노선 수 반영)')['등록차량수'].mean(), train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].value_counts()

(도보 10분거리 내 지하철역 수(환승노선 수 반영)
 0.0    587.101664
 1.0    441.430000
 2.0    689.363636
 3.0    166.000000
 Name: 등록차량수, dtype: float64,
 0.0    2223
 1.0     400
 2.0      33
 3.0       6
 Name: 도보 10분거리 내 지하철역 수(환승노선 수 반영), dtype: int64)

In [23]:
## 지하철역의 결측치가 존재하는 단지코드의 등록차량수 출력
train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()]\
.drop_duplicates(['단지코드'], keep='first')[['단지코드', '등록차량수']]

,단지코드,등록차량수
86,C1312,359.0
90,C1874,62.0
136,C1424,179.0
139,C2100,671.0
163,C2520,386.0
168,C1616,98.0
190,C1704,131.0
207,C2258,78.0
339,C1068,510.0
345,C1983,253.0


#### 지하철 역 수에 대해 전체적으로 정보를 출력

In [19]:
print(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==0]['등록차량수'].describe())
print(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==0]['단지코드'].value_counts())
print(len(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==0]['단지코드'].value_counts()))
print()

print(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==1]['등록차량수'].describe())
print(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==1]['단지코드'].value_counts())
print(len(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==1]['단지코드'].value_counts()))
print()

print(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==2]['등록차량수'].describe())
print(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==2]['단지코드'].value_counts())
print(len(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==2]['단지코드'].value_counts()))
print()

print(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==3]['등록차량수'].describe())
print(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==3]['단지코드'].value_counts())
print(len(train1[train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)']==3]['단지코드'].value_counts()))
print()

count    2223.000000
mean      587.101664
std       417.936131
min        16.000000
25%       293.000000
50%       509.000000
75%       787.000000
max      2550.000000
Name: 등록차량수, dtype: float64
C2132    47
C1790    33
C2597    32
C2289    24
C2310    23
         ..
C2490     2
C1768     2
C2340     1
C2437     1
C2188     1
Name: 단지코드, Length: 348, dtype: int64
348

count     400.000000
mean      441.430000
std       389.544551
min        13.000000
25%       132.000000
50%       313.000000
75%       649.000000
max      1668.000000
Name: 등록차량수, dtype: float64
C1439    48
C1859    40
C2034    29
C1109    22
C2038    20
C2135    20
C2621    18
C1177    16
C2393    13
C1652    12
C2659    11
C1407    11
C1338    10
C1689    10
C1343     9
C2173     8
C1129     8
C2420     7
C2596     7
C1743     7
C1032     6
C2357     6
C1945     6
C1911     6
C1706     5
C1783     5
C1585     4
C1916     4
C1663     4
C2319     4
C2561     4
C1143     3
C1613     3
C2020     3
C2059     3
C2470     3
C

-> 아무래도 표본이 0과 1에 치중이 되어있다보니 등록차량수는 정확한 지표로 사용하기 힘들 것 같다.

-> 일반적으로 지하철 역이 주변에 3개나 있으면 차량을 많이 가지고 있지 않을 것이다.

or 역이 주변에 많기 때문에 집값도 비쌀 것이고 그만큼 차도 많을 것이다.

#### 그냥 0으로 몰아넣자

In [121]:
train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)'] = \
train1['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].fillna(0)

test1['도보 10분거리 내 지하철역 수(환승노선 수 반영)'] = \
test1['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].fillna(0)

train1.info(), test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2869 entries, 0 to 2868
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   index                         2869 non-null   int64  
 1   단지코드                          2869 non-null   object 
 2   총세대수                          2869 non-null   int64  
 3   임대건물구분                        2869 non-null   object 
 4   지역                            2869 non-null   object 
 5   공급유형                          2869 non-null   object 
 6   전용면적                          2869 non-null   float64
 7   전용면적별세대수                      2869 non-null   int64  
 8   공가수                           2869 non-null   float64
 9   자격유형                          2869 non-null   object 
 10  임대보증금                         2869 non-null   float64
 11  임대료                           2869 non-null   float64
 12  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2869 non-null   float64
 13  도보 

(None, None)

In [33]:
test1[test1['자격유형'].isna()]

## 196 경상남도 국민임대 = A
## 258 강원도 영구임대 = C

,index,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
196,196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000.0,37470.0,0.0,2.0,840.0
258,258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000.0,44770.0,0.0,2.0,173.0


In [122]:
test1.iloc[196, 9] = 'A'
test1.iloc[258, 9] = 'C'

test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   index                         1008 non-null   int64  
 1   단지코드                          1008 non-null   object 
 2   총세대수                          1008 non-null   int64  
 3   임대건물구분                        1008 non-null   object 
 4   지역                            1008 non-null   object 
 5   공급유형                          1008 non-null   object 
 6   전용면적                          1008 non-null   float64
 7   전용면적별세대수                      1008 non-null   int64  
 8   공가수                           1008 non-null   float64
 9   자격유형                          1008 non-null   object 
 10  임대보증금                         1008 non-null   float64
 11  임대료                           1008 non-null   float64
 12  도보 10분거리 내 지하철역 수(환승노선 수 반영)  1008 non-null   float64
 13  도보 

In [74]:
pd.set_option('display.max_rows', 30)
# test1

### LabelEncoding

등록 차량수와 연관이 깊어보이는 파라미터만 사용하자

In [26]:
print(train1['임대건물구분'].value_counts())
print(train1.groupby('임대건물구분')['등록차량수'].mean(), '\n')

print(train1['지역'].value_counts())
print(train1.groupby('지역')['등록차량수'].mean(), '\n')

print(train1['공급유형'].value_counts())
print(train1.groupby('공급유형')['등록차량수'].mean(), '\n')

print(train1['자격유형'].value_counts())
print(train1.groupby('자격유형')['등록차량수'].mean(), '\n')

## 임대건물구분, 지역, 자격유형이 적절해보임. 공급유형은 counting이 적은 범주의 수치가 
## 높게나오는 경향이 있어 뭔가 정확해보이지 않음

아파트    2307
상가      562
Name: 임대건물구분, dtype: int64
임대건물구분
상가     158.393238
아파트    645.556567
Name: 등록차량수, dtype: float64 

경기도        595
경상남도       363
부산광역시      328
대전광역시      266
강원도        187
충청북도       163
충청남도       158
전라남도       147
광주광역시      141
경상북도       111
전라북도       109
대구광역시      102
제주특별자치도     91
서울특별시       50
울산광역시       34
세종특별자치시     24
Name: 지역, dtype: int64
지역
강원도         388.245989
경기도         797.400000
경상남도        409.685950
경상북도        613.855856
광주광역시       731.858156
대구광역시       873.519608
대전광역시       535.394737
부산광역시       316.545732
서울특별시       682.780000
세종특별자치시    1785.708333
울산광역시       337.117647
전라남도        509.680272
전라북도        482.422018
제주특별자치도     333.219780
충청남도        239.158228
충청북도        563.184049
Name: 등록차량수, dtype: float64 

국민임대         1727
임대상가          562
행복주택          202
공공임대(10년)     175
영구임대          152
공공임대(50년)      31
공공분양            7
공공임대(분납)        7
공공임대(5년)        3
장기전세            3
Name: 공급유형, dtype: int64
공급유형
공공

```
['상가' '아파트']
['상가' '아파트']
['강원도' '경기도' '경상남도' '경상북도' '광주광역시' '대구광역시' '대전광역시' '부산광역시' '서울특별시'
 '세종특별자치시' '울산광역시' '전라남도' '전라북도' '제주특별자치도' '충청남도' '충청북도']
['강원도' '경기도' '경상남도' '경상북도' '광주광역시' '대구광역시' '대전광역시' '부산광역시' '세종특별자치시'
 '울산광역시' '전라남도' '전라북도' '제주특별자치도' '충청남도' '충청북도']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O']
['A' 'C' 'D' 'E' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N']
['공공분양' '공공임대(10년)' '공공임대(50년)' '공공임대(5년)' '공공임대(분납)' '국민임대' '영구임대' '임대상가'
 '장기전세' '행복주택']
['공공임대(10년)' '공공임대(50년)' '공공임대(분납)' '국민임대' '영구임대' '임대상가' '행복주택']
```

train dataset과 test dataset간의 차이가 있어서 같은 label로 인코딩이 되지 않아 fit을 한번만 사용해주어야 함.

In [123]:
## 우선 모두 진행
from sklearn.preprocessing import LabelEncoder

cate = ['임대건물구분', '지역', '자격유형', '공급유형']

encoder = LabelEncoder()

for one in cate:
    encoder.fit(train1[one])
    train1[one] = encoder.transform(train1[one])
    print(encoder.classes_)
    
    test1[one] = encoder.transform(test1[one])
    print(encoder.classes_)

['상가' '아파트']
['상가' '아파트']
['강원도' '경기도' '경상남도' '경상북도' '광주광역시' '대구광역시' '대전광역시' '부산광역시' '서울특별시'
 '세종특별자치시' '울산광역시' '전라남도' '전라북도' '제주특별자치도' '충청남도' '충청북도']
['강원도' '경기도' '경상남도' '경상북도' '광주광역시' '대구광역시' '대전광역시' '부산광역시' '서울특별시'
 '세종특별자치시' '울산광역시' '전라남도' '전라북도' '제주특별자치도' '충청남도' '충청북도']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O']
['공공분양' '공공임대(10년)' '공공임대(50년)' '공공임대(5년)' '공공임대(분납)' '국민임대' '영구임대' '임대상가'
 '장기전세' '행복주택']
['공공분양' '공공임대(10년)' '공공임대(50년)' '공공임대(5년)' '공공임대(분납)' '국민임대' '영구임대' '임대상가'
 '장기전세' '행복주택']


In [125]:
train1.info(), test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2869 entries, 0 to 2868
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   index                         2869 non-null   int64  
 1   단지코드                          2869 non-null   object 
 2   총세대수                          2869 non-null   int64  
 3   임대건물구분                        2869 non-null   int32  
 4   지역                            2869 non-null   int32  
 5   공급유형                          2869 non-null   int32  
 6   전용면적                          2869 non-null   float64
 7   전용면적별세대수                      2869 non-null   int64  
 8   공가수                           2869 non-null   float64
 9   자격유형                          2869 non-null   int32  
 10  임대보증금                         2869 non-null   float64
 11  임대료                           2869 non-null   float64
 12  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2869 non-null   float64
 13  도보 

(None, None)

In [129]:
cate = ['임대건물구분', '지역', '자격유형', '공급유형']

for one in cate:
    print(train1[one].value_counts().sort_index())
    print('------------------------------------------------')
    print(test1[one].value_counts().sort_index())
    print('================================================')

0     562
1    2307
Name: 임대건물구분, dtype: int64
------------------------------------------------
0    177
1    831
Name: 임대건물구분, dtype: int64
0     187
1     595
2     363
3     111
4     141
5     102
6     266
7     328
8      50
9      24
10     34
11    147
12    109
13     91
14    158
15    163
Name: 지역, dtype: int64
------------------------------------------------
0     106
1     251
2      85
3      58
4      46
5      36
6      52
7      88
9      19
10     73
11     37
12     39
13     26
14     41
15     51
Name: 지역, dtype: int64
0     1729
1       21
2       95
3      569
4       37
5        3
6        9
7      155
8       49
9      103
10      33
11      33
12       2
13      30
14       1
Name: 자격유형, dtype: int64
------------------------------------------------
0     565
2      33
3     180
4      10
6       1
7      92
8       7
9      80
10     16
11     12
12      2
13     10
Name: 자격유형, dtype: int64
0       7
1     175
2      31
3       3
4       7
5    1727
6     152


```
['상가' '아파트']
['상가' '아파트']
['강원도' '경기도' '경상남도' '경상북도' '광주광역시' '대구광역시' '대전광역시' '부산광역시' '서울특별시'
 '세종특별자치시' '울산광역시' '전라남도' '전라북도' '제주특별자치도' '충청남도' '충청북도']
['강원도' '경기도' '경상남도' '경상북도' '광주광역시' '대구광역시' '대전광역시' '부산광역시' '세종특별자치시'
 '울산광역시' '전라남도' '전라북도' '제주특별자치도' '충청남도' '충청북도']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O']
['A' 'C' 'D' 'E' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N']
['공공분양' '공공임대(10년)' '공공임대(50년)' '공공임대(5년)' '공공임대(분납)' '국민임대' '영구임대' '임대상가'
 '장기전세' '행복주택']
['공공임대(10년)' '공공임대(50년)' '공공임대(분납)' '국민임대' '영구임대' '임대상가' '행복주택']
```

sklearn의 LabelEncoder로 시도해본 결과, train dataset과 test dataset간의 차이가 있어서 같은 label로 인코딩이 되지 않아 fit을 한번만 사용해주어야 함.

### 수치 정규화

0 ~ 20 사이로 정규화

In [131]:
## 시험삼아
scale_test = train1['임대료'].copy()

scale_test = scale_test.values.reshape(-1, 1)

In [132]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 20))   # 0 ~ 20 사이로 정규화
scaler.fit(scale_test)

test_scaled = scaler.transform(scale_test)

min(test_scaled), max(test_scaled), test_scaled.reshape(-1)

(array([0.]),
 array([20.]),
 array([0.07499791, 0.10236553, 0.10236553, ..., 0.14216681, 0.14216681,
        0.15203227]))

In [43]:
descr = ['공가수', '도보 10분거리 내 버스정류장 수', '단지내주차면수']

for one in descr:
    print(train1[one].describe(), '\n')

count    2869.000000
mean       12.915999
std        10.679931
min         0.000000
25%         4.000000
50%        11.000000
75%        20.000000
max        55.000000
Name: 공가수, dtype: float64 

count    2869.000000
mean        3.709655
std         2.676486
min         0.000000
25%         2.000000
50%         3.000000
75%         4.000000
max        20.000000
Name: 도보 10분거리 내 버스정류장 수, dtype: float64 

count    2869.000000
mean      591.934472
std       391.292702
min        13.000000
25%       277.000000
50%       500.000000
75%       812.000000
max      1798.000000
Name: 단지내주차면수, dtype: float64 



In [135]:
from sklearn.preprocessing import MinMaxScaler

scal = ['총세대수', '전용면적', '전용면적별세대수', '공가수', '임대보증금', '임대료', '단지내주차면수']
scaler = MinMaxScaler(feature_range=(0, 20))   # 0 ~ 20 사이로 정규화

for one in scal:
    scale_train = train1[one].values.reshape(-1, 1)
    scale_test = test1[one].values.reshape(-1, 1)
    
    scaler.fit(scale_train)
    train1[one] = scaler.transform(scale_train).reshape(-1)
    test1[one] = scaler.transform(scale_test).reshape(-1)

In [139]:
descr = ['총세대수', '전용면적', '전용면적별세대수', '공가수', '임대보증금', '임대료', '단지내주차면수']

for one in descr:
    print(train1[one].describe(), '\n')
    
print('=============================================')

for one in descr:
    print(test1[one].describe(), '\n')

count    2869.000000
mean        6.779341
std         4.073919
min         0.000000
25%         3.839496
50%         5.892998
75%         8.489378
max        20.000000
Name: 총세대수, dtype: float64 

count    2869.000000
mean        1.114314
std         1.123803
min         0.000000
25%         0.682575
50%         0.953783
75%         1.346578
max        20.000000
Name: 전용면적, dtype: float64 

count    2869.000000
mean        1.091201
std         1.430124
min         0.000000
25%         0.139485
50%         0.633047
75%         1.512876
max        20.000000
Name: 전용면적별세대수, dtype: float64 

count    2869.000000
mean        4.696727
std         3.883611
min         0.000000
25%         1.454545
50%         4.000000
75%         7.272727
max        20.000000
Name: 공가수, dtype: float64 

count    2869.000000
mean        0.464067
std         0.557809
min         0.000000
25%         0.192183
50%         0.345649
75%         0.569931
max        20.000000
Name: 임대보증금, dtype: float64 

count    28

In [141]:
## 전용면적이 (-)인 곳이 있네? 삭제
test1 = test1.loc[test1['전용면적']>0, :].reset_index()
test1.iloc[233]

level_0                               234
index                                 234
단지코드                                C1812
총세대수                              7.82848
임대건물구분                                  0
지역                                     10
공급유형                                    7
전용면적                            0.0157679
전용면적별세대수                                0
공가수                                     8
자격유형                                    3
임대보증금                           0.0112982
임대료                              0.389549
도보 10분거리 내 지하철역 수(환승노선 수 반영)            0
도보 10분거리 내 버스정류장 수                      4
단지내주차면수                           1.92717
Name: 233, dtype: object

In [143]:
test1.loc[test1['전용면적']<0, :]

,level_0,index,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수


### 여기까지 전처리한 부분 우선 저장

In [149]:
train1.to_csv('parking_data_preprocessing_train.csv', index=False)
test1.to_csv('parking_data_preprocessing_test.csv', index=False)

import os
print(os.listdir(os.getcwd()))

['.ipynb_checkpoints', 'parking_data_preprocessing_test.csv', 'parking_data_preprocessing_train.csv', 'Parking_demand_07_12(1).ipynb', 'Parking_demand_07_12(2).ipynb', 'Parking_demand_07_14.ipynb', 'Parking_demand_07_16.ipynb', 'Parking_demand_07_17.ipynb', 'Parking_demand_data_error.ipynb', 'second_rf_0712.csv']
